In [171]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/content/.config/.last_survey_prompt.yaml
/content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
/content/.config/active_config
/content/.config/default_configs.db
/content/.config/.last_opt_in_prompt.yaml
/content/.config/gce
/content/.config/config_sentinel
/content/.config/.last_update_check.json
/content/.config/logs/2024.11.21/14.25.30.368557.log
/content/.config/logs/2024.11.21/14.24.55.851126.log
/content/.config/logs/2024.11.21/14.25.31.482956.log
/content/.config/logs/2024.11.21/14.25.17.504089.log
/content/.config/logs/2024.11.21/14.25.44.623490.log
/content/.config/logs/2024.11.21/14.25.43.887178.log
/content/.config/configurations/config_default
/content/sample_data/anscombe.json
/content/sample_data/README.md
/content/sample_data/mnist_train_small.csv
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/california_housing_train.csv


In [172]:
# tensorflow hub
import tensorflow_hub as hub
# tensor flow module
import tensorflow as tf
import tensorflow_probability as tfp

# matplotlib
from matplotlib import colors
from matplotlib import pyplot as plt

# word vectorizor
# first converts the text into a matrix of word counts
# then transforms these counts by normalizing them based on the term frequency
from sklearn.feature_extraction.text import TfidfVectorizer

# used to create word encoders
from sklearn import preprocessing

# Import Data

read in data to pandas dataframes

In [173]:
train_df = pd.read_csv("/train.csv")
test_df = pd.read_csv("/test.csv")

# Exploratory

In [174]:
train_df.groupby("target")["id"].nunique()

,id
target,
0,4342
1,3271


In [175]:
print(len(test_df.loc[test_df["location"].notnull(),"location"]))
print(len(test_df.loc[test_df["location"].notnull(),"location"].unique()))

2158
1602


In [176]:
print(len(test_df.loc[test_df["keyword"].notnull(),"keyword"]))
print(len(test_df.loc[test_df["keyword"].notnull(),"keyword"].unique()))

3237
221


# Tweet Text: Sentence Embedding

Next we can convert each of the texts into vectors using a TensorFlow universal-sentence-encoder.

In [177]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/3")
X_train_embeddings = embed(train_df["text"].values)
X_test_embeddings = embed(test_df["text"].values)

# Tensor Flow Naiver Bayes

Create numpy matricies for the features in the training and test set.

In [178]:
X_train_matrix = X_train_embeddings['outputs'].numpy()

In [179]:
X_test_matrix = X_test_embeddings['outputs'].numpy()

The target is y variable for my model.

In [180]:
y_train = tf.constant(train_df["target"])

First create a python object that can fit and predict a naive bayes classifier

In [181]:
class TFNaiveBayesClassifier:
    dist = None

    # X is the matrix containing the vectors for each sentence
    # y is the list target values in the same order as the X matrix
    def fit(self, X, y):
        unique_y = np.unique(y)  # unique target values: 0,1
        print(unique_y)
        # `points_by_class` is a list of numpy arrays, one for each class.
        # Each array contains the data points belonging to that class.
        points_by_class = []
        for c in unique_y:
            class_indices = np.where(y == c)[0]  # Get indices of data points for class c
            class_data = X.iloc[class_indices, :].values  # Get data for those indices
            points_by_class.append(class_data)  # Add data of class c into the point_by_class list

        mean_list = []
        var_list = []
        for i in range(0, len(points_by_class)):
            mean_var, var_var = tf.nn.moments(tf.constant(points_by_class[i]), axes=[0])
            mean_list.append(mean_var)
            var_list.append(var_var)
        mean = tf.stack(mean_list, 0)
        var = tf.stack(var_list, 0)
        # Create a 3x2 univariate normal distribution with the
        # known mean and variance
        self.dist = tfp.distributions.Normal(loc=mean, scale=tf.sqrt(var))

    def predict(self, X):
        assert self.dist is not None
        nb_classes, nb_features = map(int, self.dist.scale.shape)

        # uniform priors
        priors = np.log(np.array([1. / nb_classes] * nb_classes))
        # Convert priors to tf.float32 to match cond_probs
        priors = tf.cast(priors, dtype=tf.float32)

        # Conditional probabilities log P(x|c)
        # (nb_samples, nb_classes, nb_features)
        all_log_probs = self.dist.log_prob(
            tf.reshape(
                tf.tile(X, [1, nb_classes]), [-1, nb_classes, nb_features]))
        # (nb_samples, nb_classes)
        cond_probs = tf.reduce_sum(all_log_probs, axis=2)

        # posterior log probability, log P(c) + log P(x|c)
        joint_likelihood = tf.add(priors, cond_probs)

        # normalize to get (log)-probabilities
        norm_factor = tf.reduce_logsumexp(
            joint_likelihood, axis=1, keepdims=True)

        # Compute and return the predicted class probabilities (or labels if needed)
        log_probs = joint_likelihood - norm_factor # Subtract norm_factor to normalize
        #To get the predicted labels you can use:
        # predicted_labels = tf.argmax(log_probs, axis=1) #Returns the index with the maximum log probability
        return log_probs # or predicted_labels, depending on what you need

Here initialize naive bayes model and fit it using the training data

In [182]:
tf_nb = TFNaiveBayesClassifier()
tf_nb.fit(pd.DataFrame(X_train_matrix),
          y_train)

[0 1]


predict probability of each target values in the test set

In [183]:
y_pred = tf_nb.predict(X_test_matrix)

Create a dataframe containing the probability of each target given the text in each tweet.

In [184]:
predProbGivenText_df = pd.DataFrame(y_pred.numpy())
predProbGivenText_df.head()

,0,1
0,-0.223999,-1.605957
1,-20.051208,0.000000
2,-7.355286,-0.000610
3,-28.114563,0.000000
4,-39.491333,0.000000


# Keyword Naive Bayes

Let's look at our unique key words.

In [185]:
uniq_keywords = train_df["keyword"].unique()[1:]
print(len(uniq_keywords))
print(uniq_keywords)

221
['ablaze' 'accident' 'aftershock' 'airplane%20accident' 'ambulance'
 'annihilated' 'annihilation' 'apocalypse' 'armageddon' 'army' 'arson'
 'arsonist' 'attack' 'attacked' 'avalanche' 'battle' 'bioterror'
 'bioterrorism' 'blaze' 'blazing' 'bleeding' 'blew%20up' 'blight'
 'blizzard' 'blood' 'bloody' 'blown%20up' 'body%20bag' 'body%20bagging'
 'body%20bags' 'bomb' 'bombed' 'bombing' 'bridge%20collapse'
 'buildings%20burning' 'buildings%20on%20fire' 'burned' 'burning'
 'burning%20buildings' 'bush%20fires' 'casualties' 'casualty'
 'catastrophe' 'catastrophic' 'chemical%20emergency' 'cliff%20fall'
 'collapse' 'collapsed' 'collide' 'collided' 'collision' 'crash' 'crashed'
 'crush' 'crushed' 'curfew' 'cyclone' 'damage' 'danger' 'dead' 'death'
 'deaths' 'debris' 'deluge' 'deluged' 'demolish' 'demolished' 'demolition'
 'derail' 'derailed' 'derailment' 'desolate' 'desolation' 'destroy'
 'destroyed' 'destruction' 'detonate' 'detonation' 'devastated'
 'devastation' 'disaster' 'displaced' 'droug

I noticed that there are some keywords that are very similar to each other so I decided to manually make them the same word.

In [186]:
def replace_keywords(df_og):
    df = df_og.copy()
    df["keyword"] = df["keyword"].replace("ablaze","blaze")
    df["keyword"] = df["keyword"].replace("blazing","blaze")
    df["keyword"] = df["keyword"].replace("annihilated","annihilation")
    df["keyword"] = df["keyword"].replace("attacked","attack")
    df["keyword"] = df["keyword"].replace("bioterror","bioterrorism")
    df["keyword"] = df["keyword"].replace("blown%20up","blew%20up")
    df["keyword"] = df["keyword"].replace("bloody","blood")
    df["keyword"] = df["keyword"].replace("bleeding","blood")
    df["keyword"] = df["keyword"].replace("body%20bags","body%20bag")
    df["keyword"] = df["keyword"].replace("body%20bagging","body%20bag")
    df["keyword"] = df["keyword"].replace("bombed","bomb")
    df["keyword"] = df["keyword"].replace("bombing","bomb")
    df["keyword"] = df["keyword"].replace("burning%20buildings","buildings%20burning")
    df["keyword"] = df["keyword"].replace("buildings%20on%20fire","buildings%20burning")
    df["keyword"] = df["keyword"].replace("burned","burning")
    df["keyword"] = df["keyword"].replace("casualties","casualty")
    df["keyword"] = df["keyword"].replace("catastrophe","catastrophic")
    df["keyword"] = df["keyword"].replace("collapse","collapsed")
    df["keyword"] = df["keyword"].replace("collide","collision")
    df["keyword"] = df["keyword"].replace("collided","collision")
    df["keyword"] = df["keyword"].replace("crash","crashed")
    df["keyword"] = df["keyword"].replace("crush","crushed")
    df["keyword"] = df["keyword"].replace("dead","death")
    df["keyword"] = df["keyword"].replace("deaths","death")
    df["keyword"] = df["keyword"].replace("deluge","deluged")
    df["keyword"] = df["keyword"].replace("demolished","demolish")
    df["keyword"] = df["keyword"].replace("demolition","demolish")
    df["keyword"] = df["keyword"].replace("derailment","derail")
    df["keyword"] = df["keyword"].replace("derailed","derail")
    df["keyword"] = df["keyword"].replace("desolation","desolate")
    df["keyword"] = df["keyword"].replace("destroyed","destroy")
    df["keyword"] = df["keyword"].replace("destruction","destroy")
    df["keyword"] = df["keyword"].replace("detonate","detonation")
    df["keyword"] = df["keyword"].replace("devastated","devastation")
    df["keyword"] = df["keyword"].replace("drowned","drown")
    df["keyword"] = df["keyword"].replace("drowning","drown")
    df["keyword"] = df["keyword"].replace("electrocute","electrocuted")
    df["keyword"] = df["keyword"].replace("evacuated","evacuate")
    df["keyword"] = df["keyword"].replace("evacuation","evacuate")
    df["keyword"] = df["keyword"].replace("explode","explosion")
    df["keyword"] = df["keyword"].replace("exploded","explosion")
    df["keyword"] = df["keyword"].replace("fatality","fatalities")
    df["keyword"] = df["keyword"].replace("floods","flood")
    df["keyword"] = df["keyword"].replace("flooding","flood")
    df["keyword"] = df["keyword"].replace("bush%20fires","forest%20fire")
    df["keyword"] = df["keyword"].replace("forest%20fires","forest%20fire")
    df["keyword"] = df["keyword"].replace("hailstorm","hail")
    df["keyword"] = df["keyword"].replace("hazardous","hazard")
    df["keyword"] = df["keyword"].replace("hijacking","hijack")
    df["keyword"] = df["keyword"].replace("hijacker","hijack")
    df["keyword"] = df["keyword"].replace("hostages","hostage")
    df["keyword"] = df["keyword"].replace("injured","injury")
    df["keyword"] = df["keyword"].replace("injures","injury")
    df["keyword"] = df["keyword"].replace("inundated","inundation")
    df["keyword"] = df["keyword"].replace("mass%20murderer","mass%20murder")
    df["keyword"] = df["keyword"].replace("obliterated","obliterate")
    df["keyword"] = df["keyword"].replace("obliteration","obliterate")
    df["keyword"] = df["keyword"].replace("panicking","panic")
    df["keyword"] = df["keyword"].replace("quarantined","quarantine")
    df["keyword"] = df["keyword"].replace("rescuers","rescue")
    df["keyword"] = df["keyword"].replace("rescued","rescue")
    df["keyword"] = df["keyword"].replace("rioting","riot")
    df["keyword"] = df["keyword"].replace("dust%20storm","sandstorm")
    df["keyword"] = df["keyword"].replace("screamed","screams")
    df["keyword"] = df["keyword"].replace("screaming","screams")
    df["keyword"] = df["keyword"].replace("sirens","siren")
    df["keyword"] = df["keyword"].replace("suicide%20bomb","suicide%20bomber")
    df["keyword"] = df["keyword"].replace("suicide%20bombing","suicide%20bomber")
    df["keyword"] = df["keyword"].replace("survived","survive")
    df["keyword"] = df["keyword"].replace("survivors","survive")
    df["keyword"] = df["keyword"].replace("terrorism","terrorist")
    df["keyword"] = df["keyword"].replace("thunderstorm","thunder")
    df["keyword"] = df["keyword"].replace("traumatised","trauma")
    df["keyword"] = df["keyword"].replace("twister","tornado")
    df["keyword"] = df["keyword"].replace("typhoon","hurricane")
    df["keyword"] = df["keyword"].replace("weapons","weapon")
    df["keyword"] = df["keyword"].replace("wild%20fires","wildfire")
    df["keyword"] = df["keyword"].replace("wounded","wounds")
    df["keyword"] = df["keyword"].replace("wrecked","wreckage")
    df["keyword"] = df["keyword"].replace("wreck","wreckage")
    return(df)

In [187]:
train_df = replace_keywords(train_df)
test_df = replace_keywords(test_df)

Using the updated set of keywords let's get the probability for each target given a specific key word.

In [188]:
uniq_keywords = train_df["keyword"].unique()[1:]
kword_resArr = []
print(len(uniq_keywords))
for kword in uniq_keywords:
    kword_df = train_df.loc[train_df["keyword"] == kword,: ]
    total_kword = float(len(kword_df))
    target0_n = float(len(kword_df.loc[kword_df["target"]==0,:]))
    target1_n = float(len(kword_df.loc[kword_df["target"]==1,:]))
    kword_prob_df = pd.DataFrame({'keyword':[kword],
                                 "keywordPred0": [target0_n/total_kword],
                                 "keywordPred1": [target1_n/total_kword]})
    kword_resArr.append(kword_prob_df)
predProbGivenKeyWord_df= pd.concat(kword_resArr)
predProbGivenKeyWord_df.head()

142


,keyword,keywordPred0,keywordPred1
0,blaze,0.824074,0.175926
0,accident,0.314286,0.685714
0,aftershock,1.000000,0.000000
0,airplane%20accident,0.142857,0.857143
0,ambulance,0.473684,0.526316


# Create submission file

get probabilities given the tweet text

In [189]:
test_df["textprob0"]=predProbGivenText_df.loc[:,0].copy()
test_df["textprob1"]=predProbGivenText_df.loc[:,1].copy()
test_df.head()

,id,keyword,location,text,textprob0,textprob1
0,0,NaN,NaN,Just happened a terrible car crash,-0.223999,-1.605957
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",-20.051208,0.000000
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",-7.355286,-0.000610
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,-28.114563,0.000000
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,-39.491333,0.000000


get the probabilities given the key words. Note that if there is no key word than the probability is 50% for both targets.

In [190]:
test_df = test_df.merge(predProbGivenKeyWord_df, how='left', on="keyword")

In [191]:
test_df["keywordPred0"]=test_df["keywordPred0"].fillna(0.5)
test_df["keywordPred1"]=test_df["keywordPred1"].fillna(0.5)

Now let's calculate the probability given the text and the keyword. Then let's choose our prediction based on which target has the higher probability.

In [192]:
test_df["pred0"]=test_df["textprob0"]*test_df["keywordPred0"]
test_df["pred1"]=test_df["textprob1"]*test_df["keywordPred1"]
test_df["target"]=test_df["pred1"]>test_df["pred0"]
test_df["target"] = test_df["target"].astype(int)

Create submission file

In [193]:
submission_df = test_df.loc[:,["id","target"]]
submission_df.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [194]:
submission_df.to_csv("submission.csv",index=False)